In [3]:
import psycopg2
import pandas as pd
from datetime import timedelta

In [ ]:
!pip install psycopg2-binary

In [ ]:
# Configurações de conexão (Usar banco disponibilizado na aula ou testar com banco remoto feito para testes))
host = ''
database = ''
user = ''
password = ''

In [5]:
# Conectar ao PostgreSQL
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    print('Conexão estabelecida com sucesso!')
    
    # Criar cursor
    cursor = conn.cursor()
    
except Exception as e:
    print(f'Erro ao conectar: {e}')

Conexão estabelecida com sucesso!


In [6]:
# Consulta SQL
query = """
select nf.id, nf.data_venda::date as data, nf.valor , COALESCE(pf.cpf, pj.cnpj) AS cpf_cnpj 
from vendas.nota_fiscal nf 
left join geral.pessoa_fisica pf on pf.id = nf.id_cliente 
left join geral.pessoa_juridica pj on pj.id = nf.id_cliente
"""

# Executar consulta e criar DataFrame
df = pd.read_sql_query(query, conn)
print(f'DataFrame criado com {len(df)} registros')
df.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_6412\1641402841.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


DataFrame criado com 122888 registros


,id,data,valor,cpf_cnpj
0,1,2015-03-08,120.0,04159432301
1,2,2016-06-08,43.0,05030272356
2,3,2016-11-05,3560.0,00195990307
3,4,2016-04-21,8080.0,25885324387
4,5,2016-11-22,139.0,03270757328


In [7]:
# Data de referência para cálculo da Recência
data_referencia = df['data'].max()
print(f'Data de referência: {data_referencia}')

# Calcular métricas RFM por cliente
rfm = df.groupby('cpf_cnpj').agg({
    'data': lambda x: (data_referencia - x.max()).days,
    'id': 'count',
    'valor': 'sum'
}).reset_index()

rfm.columns = ['cpf_cnpj', 'recencia', 'frequencia', 'valor_monetario']
print(f'RFM calculado para {len(rfm)} clientes')
rfm.head()

Data de referência: 2026-01-14
RFM calculado para 15931 clientes


,cpf_cnpj,recencia,frequencia,valor_monetario
0,00000482323,843,2,759.63
1,00003290379,388,6,32250.68
2,00007517319,94,3,13414.90
3,00007527381,1182,5,12996.70
4,00008012326,7,31,141852.79


In [8]:
# Criar scores RFM (1-5)
rfm['r_score'] = pd.qcut(rfm['recencia'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequencia'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['valor_monetario'], 5, labels=[1,2,3,4,5])

# Score RFM combinado
rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

rfm.head()

,cpf_cnpj,recencia,frequencia,valor_monetario,r_score,f_score,m_score,rfm_score
0,00000482323,843,2,759.63,1,1,1,111
1,00003290379,388,6,32250.68,2,3,2,232
2,00007517319,94,3,13414.90,4,1,1,411
3,00007527381,1182,5,12996.70,1,2,1,121
4,00008012326,7,31,141852.79,5,5,5,555


In [9]:
# Segmentação simplificada
def segmentar_cliente(row):
    r, f, m = int(row['r_score']), int(row['f_score']), int(row['m_score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeões'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Clientes Leais'
    elif r >= 4 and f <= 2:
        return 'Novos Clientes'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'Em Risco'
    elif r <= 2 and f <= 2:
        return 'Perdido'
    else:
        return 'Clientes Potenciais'


In [10]:
rfm['segmento'] = rfm.apply(segmentar_cliente, axis=1)

In [11]:
rfm.head()

,cpf_cnpj,recencia,frequencia,valor_monetario,r_score,f_score,m_score,rfm_score,segmento
0,00000482323,843,2,759.63,1,1,1,111,Perdido
1,00003290379,388,6,32250.68,2,3,2,232,Clientes Potenciais
2,00007517319,94,3,13414.90,4,1,1,411,Novos Clientes
3,00007527381,1182,5,12996.70,1,2,1,121,Perdido
4,00008012326,7,31,141852.79,5,5,5,555,Campeões


In [12]:
# Análise dos segmentos
segmentos = rfm.groupby('segmento').agg({
    'cpf_cnpj': 'count',
    'recencia': 'mean',
    'frequencia': 'mean',
    'valor_monetario': 'mean'
}).round(2)

segmentos.columns = ['qtd_clientes', 'recencia_media', 'frequencia_media', 'valor_medio']
segmentos['percentual'] = (segmentos['qtd_clientes'] / len(rfm) * 100).round(1)

print('Análise por Segmento:')
segmentos.sort_values('qtd_clientes', ascending=False)

Análise por Segmento:


,qtd_clientes,recencia_media,frequencia_media,valor_medio,percentual
segmento,,,,,
Perdido,3438,876.42,3.67,26771.97,21.6
Clientes Leais,3108,183.30,9.17,62796.95,19.5
Clientes Potenciais,2777,294.55,5.62,28909.14,17.4
Campeões,2436,81.47,12.78,85109.23,15.3
Em Risco,2368,645.95,11.63,72674.17,14.9
Novos Clientes,1804,90.19,4.15,30438.23,11.3


## Mais passo a passo

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122888 entries, 0 to 122887
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        122888 non-null  int64  
 1   data      122888 non-null  object 
 2   valor     122888 non-null  float64
 3   cpf_cnpj  122888 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.8+ MB


In [14]:
data_mais_atual = df['data'].max() + timedelta(days=1)
data_mais_atual

datetime.date(2026, 1, 15)

In [15]:
df_agregado = df.groupby('cpf_cnpj').agg({'data': lambda x: (data_mais_atual - x.max()).days,
                                                   'id': 'count',
                                                   'valor': 'sum'})

In [16]:
df_agregado

,data,id,valor
cpf_cnpj,,,
00000482323,844,2,759.63
00003290379,389,6,32250.68
00007517319,95,3,13414.90
00007527381,1183,5,12996.70
00008012326,8,31,141852.79
...,...,...,...
99859653372,55,6,23142.47
99882426387,184,7,77913.55
99913321387,622,3,22266.84


In [17]:
df_agregado.shape

(15931, 3)

In [18]:
df['cpf_cnpj'].nunique()

15931

In [19]:
df_agregado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15931 entries, 00000482323 to 99933667300
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   data    15931 non-null  int64  
 1   id      15931 non-null  int64  
 2   valor   15931 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 497.8+ KB


In [20]:
df_agregado.rename(columns={'data': 'recencia', 
                            'id': 'frequencia', 
                            'valor': 'monetario'}, inplace=True)

In [21]:
df_agregado

,recencia,frequencia,monetario
cpf_cnpj,,,
00000482323,844,2,759.63
00003290379,389,6,32250.68
00007517319,95,3,13414.90
00007527381,1183,5,12996.70
00008012326,8,31,141852.79
...,...,...,...
99859653372,55,6,23142.47
99882426387,184,7,77913.55
99913321387,622,3,22266.84


In [22]:
r_groups = pd.qcut(df_agregado['recencia'], q=5, labels=range(1,6))
r_groups

cpf_cnpj
00000482323    5
00003290379    4
00007517319    2
00007527381    5
00008012326    1
              ..
99859653372    1
99882426387    2
99913321387    4
99927110359    4
99933667300    2
Name: recencia, Length: 15931, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [23]:
f_groups = pd.qcut(df_agregado['frequencia'], q=5, labels=range(1,6))
f_groups

cpf_cnpj
00000482323    1
00003290379    3
00007517319    1
00007527381    2
00008012326    5
              ..
99859653372    3
99882426387    3
99913321387    1
99927110359    5
99933667300    1
Name: frequencia, Length: 15931, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [24]:
m_groups = pd.qcut(df_agregado['monetario'], q=5, labels=range(1,6))
m_groups

cpf_cnpj
00000482323    1
00003290379    2
00007517319    1
00007527381    1
00008012326    5
              ..
99859653372    1
99882426387    5
99913321387    1
99927110359    5
99933667300    1
Name: monetario, Length: 15931, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [25]:
df_agregado['R'] = r_groups
df_agregado['F'] = f_groups
df_agregado['M'] = m_groups

In [26]:
df_agregado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15931 entries, 00000482323 to 99933667300
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   recencia    15931 non-null  int64   
 1   frequencia  15931 non-null  int64   
 2   monetario   15931 non-null  float64 
 3   R           15931 non-null  category
 4   F           15931 non-null  category
 5   M           15931 non-null  category
dtypes: category(3), float64(1), int64(2)
memory usage: 1.0+ MB


In [27]:
df_agregado['R' ] = df_agregado['R'].astype(int)
df_agregado['F'] = df_agregado['F'].astype(int)
df_agregado['M'] = df_agregado['M'].astype(int)

In [28]:
df_agregado['R'].value_counts()

R
1    3208
4    3184
3    3183
5    3179
2    3177
Name: count, dtype: int64

In [29]:
df_agregado['F'].value_counts()

F
3    4482
1    4036
4    2642
5    2409
2    2362
Name: count, dtype: int64

In [30]:
df_agregado['M'].value_counts()

M
1    3187
2    3186
5    3186
3    3186
4    3186
Name: count, dtype: int64

In [31]:
df_agregado['RFM_Score'] = df_agregado['R'] + df_agregado['F'] + df_agregado['M']
df_agregado['RFM_Segment'] = df_agregado['R'].astype(str) + df_agregado['F'].astype(str) + df_agregado['M'].astype(str)

In [32]:
df_agregado[['R', 'F', 'M', 'RFM_Score', 'RFM_Segment']].head()

,R,F,M,RFM_Score,RFM_Segment
cpf_cnpj,,,,,
00000482323,5,1,1,7,511
00003290379,4,3,2,9,432
00007517319,2,1,1,4,211
00007527381,5,2,1,8,521
00008012326,1,5,5,11,155


In [33]:
def classify_by_score(score):
    if score >= 11:
        return 'Alto Valor'
    elif score >= 7:
        return 'Médio Valor'
    else:
        return 'Baixo Valor'

def classify_by_segment(segment):
    r, f, m = int(segment[0]), int(segment[1]), int(segment[2])
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeões'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Clientes Fiéis'
    elif r >= 3 and f >= 1 and m >= 3:
        return 'Clientes Potenciais'
    elif r >= 3 and f >= 1 and m >= 1:
        return 'Novos Clientes'
    elif r >= 1 and f >= 1 and m >= 1:
        return 'Clientes em Risco'
    else:
        return 'Perdidos'

In [34]:
df_agregado['Classe_Score'] = df_agregado['RFM_Score'].apply(classify_by_score)
df_agregado['Classe_Segmento'] = df_agregado['RFM_Segment'].apply(classify_by_segment)

In [35]:
df_agregado

,recencia,frequencia,monetario,R,F,M,RFM_Score,RFM_Segment,Classe_Score,Classe_Segmento
cpf_cnpj,,,,,,,,,,
00000482323,844,2,759.63,5,1,1,7,511,Médio Valor,Novos Clientes
00003290379,389,6,32250.68,4,3,2,9,432,Médio Valor,Novos Clientes
00007517319,95,3,13414.90,2,1,1,4,211,Baixo Valor,Clientes em Risco
00007527381,1183,5,12996.70,5,2,1,8,521,Médio Valor,Novos Clientes
00008012326,8,31,141852.79,1,5,5,11,155,Alto Valor,Clientes em Risco
...,...,...,...,...,...,...,...,...,...,...
99859653372,55,6,23142.47,1,3,1,5,131,Baixo Valor,Clientes em Risco
99882426387,184,7,77913.55,2,3,5,10,235,Médio Valor,Clientes em Risco
99913321387,622,3,22266.84,4,1,1,6,411,Baixo Valor,Novos Clientes


In [36]:
print("Distribuição por Classe Score:")
print(df_agregado['Classe_Score'].value_counts())
print("\nDistribuição por Classe Segmento:")
print(df_agregado['Classe_Segmento'].value_counts())

Distribuição por Classe Score:
Classe_Score
Médio Valor    8445
Alto Valor     4457
Baixo Valor    3029
Name: count, dtype: int64

Distribuição por Classe Segmento:
Classe_Segmento
Clientes em Risco      6385
Novos Clientes         4381
Clientes Fiéis         2985
Clientes Potenciais    1114
Campeões               1066
Name: count, dtype: int64


In [37]:
def acao_por_segmento(classe):
    if classe == 'Campeões':
        return 'Priorizar: Oferecer benefícios exclusivos e programas VIP para maximizar retenção e valor.'
    elif classe == 'Clientes Fiéis':
        return 'Manter: Enviar comunicações regulares e ofertas de upsell para fortalecer o relacionamento.'
    elif classe == 'Clientes Potenciais':
        return 'Converter: Incentivar compras mais frequentes com promoções personalizadas e cross-sell.'
    elif classe == 'Novos Clientes':
        return 'Integrar: Bem-vindo e orientações para construir lealdade desde o início.'
    elif classe == 'Clientes em Risco':
        return 'Reativar: Campanhas de win-back com descontos e lembretes para recuperar interesse.'
    else:
        return 'Avaliar: Considerar se vale a pena investir em reativação ou focar em segmentos mais promissores.'

In [38]:
df_agregado['Acao'] = df_agregado['Classe_Segmento'].apply(acao_por_segmento)

In [39]:
df_agregado.sample(10)

,recencia,frequencia,monetario,R,F,M,RFM_Score,RFM_Segment,Classe_Score,Classe_Segmento,Acao
cpf_cnpj,,,,,,,,,,,
04600090365,12,8,50566.60,1,4,3,8,143,Médio Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
04210177326,338,6,56223.38,3,3,4,10,334,Médio Valor,Clientes Fiéis,Manter: Enviar comunicações regulares e oferta...
19640243000112,27,5,58226.76,1,2,4,7,124,Médio Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
10674917324,406,5,34339.84,4,2,2,8,422,Médio Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...
89711980363,735,7,45058.87,5,3,3,11,533,Alto Valor,Clientes Fiéis,Manter: Enviar comunicações regulares e oferta...
08460359360,35,5,43215.71,1,2,3,6,123,Baixo Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
01274179319,111,6,63601.36,2,3,4,9,234,Médio Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
63383442000184,1200,5,31469.04,5,2,2,9,522,Médio Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...
02976348367,330,7,54552.44,3,3,4,10,334,Médio Valor,Clientes Fiéis,Manter: Enviar comunicações regulares e oferta...


In [40]:
df_agregado

,recencia,frequencia,monetario,R,F,M,RFM_Score,RFM_Segment,Classe_Score,Classe_Segmento,Acao
cpf_cnpj,,,,,,,,,,,
00000482323,844,2,759.63,5,1,1,7,511,Médio Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...
00003290379,389,6,32250.68,4,3,2,9,432,Médio Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...
00007517319,95,3,13414.90,2,1,1,4,211,Baixo Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
00007527381,1183,5,12996.70,5,2,1,8,521,Médio Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...
00008012326,8,31,141852.79,1,5,5,11,155,Alto Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
...,...,...,...,...,...,...,...,...,...,...,...
99859653372,55,6,23142.47,1,3,1,5,131,Baixo Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
99882426387,184,7,77913.55,2,3,5,10,235,Médio Valor,Clientes em Risco,Reativar: Campanhas de win-back com descontos ...
99913321387,622,3,22266.84,4,1,1,6,411,Baixo Valor,Novos Clientes,Integrar: Bem-vindo e orientações para constru...


In [ ]:
from enviar_email import EnviarEmail

# Configurações de email (Configurar uso de credenciais próprias)
remetente = ''
senha = ''
destinatario = ''
servidor_smtp = 'smtp.gmail.com'
porta_smtp = 587

# Criar instância da classe
email_sender = EnviarEmail(servidor_smtp, porta_smtp, remetente, senha)

# Enviar relatório RFM
email_sender.enviar(
    assunto='Relatório RFM - Análise de Clientes',
    df=df_agregado.head(15),
    destinatario=destinatario
)

print('Email enviado com sucesso!')


Email enviado com sucesso!
Email enviado com sucesso!


In [ ]:
from openai import OpenAI

# Substitua 'your-openai-api-key' pela sua chave da API do OpenAI
client = OpenAI(api_key='')

# Preparar resumo dos dados para enviar
resumo = f"""
Análise RFM dos Clientes:

Estatísticas Descritivas:
{df_agregado[['recencia', 'frequencia', 'monetario']].describe()}

Distribuição dos Scores RFM:
{df_agregado['RFM_Score'].value_counts().sort_index()}

Distribuição por Classe Score:
{df_agregado['Classe_Score'].value_counts()}

Distribuição por Classe Segmento:
{df_agregado['Classe_Segmento'].value_counts()}

Amostra de Ações Recomendadas:
{df_agregado[['Classe_Segmento', 'Acao']].drop_duplicates()}
"""

# Prompt para o ChatGPT
prompt = f"Analise os dados RFM abaixo e forneça um parecer detalhado com recomendações estratégicas para melhorar o negócio baseado nesses insights. {resumo}"

# Chamada para a API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Você é um especialista em análise de dados e marketing."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=1000
)

# Exibir a resposta
print(response.choices[0].message.content)

Com base nos dados RFM fornecidos, podemos extrair insights valiosos para melhorar a estratégia de marketing e vendas da empresa. Abaixo estão as análises detalhadas e recomendações estratégicas para cada aspecto do RFM:

1. Análise de Recência, Frequência e Valor Monetário:
   - Recência (tempo desde a última compra): A média de recência é de aproximadamente 396 dias, o que indica que a empresa pode precisar trabalhar para reduzir esse tempo para garantir a fidelização dos clientes.
   - Frequência (número de compras): A média de frequência é de 7,7 compras por cliente, sugerindo que os clientes estão engajados, mas sempre há espaço para aumentar a frequência de compras.
   - Valor Monetário (receita total): A média de valor monetário é de 50.331,10, o que mostra um bom ticket médio. Porém, é possível segmentar os clientes com base no valor de compras.

2. Distribuição dos Scores RFM e Classes de Score:
   - A maioria dos clientes está concentrada nos níveis de pontuação média, o que 